In [ ]:

import math
import heapq


class RailRoadOptimizer:
    def __init__(self, file_path, output_path):
        self.file_path = file_path
        self.output_path = output_path
        self.elements = {}
        self.route_points = []
        self.graph = {}
        self.path_order = []
        self.path_top = []

    def read_input(self):
        with open(self.file_path, 'r') as f:
            lines = f.readlines()

        section = None
        for line in lines:
            line = line.split('--')[0].strip()  # Убираем комментарии
            if not line:
                continue

            if line == "DATA":
                section = "DATA"
            elif line == "ROUTE":
                section = "ROUTE"
            elif line == "/":
                section = None
            elif section == "DATA":
                self.parse_data_block(line)
            elif section == "ROUTE":
                self.parse_route_block(line)

    def parse_data_block(self, line):
        parts = line.split()
        if len(parts) != 3:
            return
        block_type, quantity, cost = parts
        self.elements[block_type] = (int(quantity), float(cost))

    def parse_route_block(self, line):
        parts = line.split()
        if len(parts) != 3:
            return
        x, y, value = map(float, parts)
        self.route_points.append((x, y, value))

    def calculate_distance(self, point1, point2):
        return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

    def build_graph(self):
        n = len(self.route_points)
        for i in range(n):
            self.graph[i] = []
            for j in range(n):
                if i != j:
                    dist = self.calculate_distance(self.route_points[i], self.route_points[j])
                    self.graph[i].append((j, dist))

    def dijkstra(self, start):
        n = len(self.route_points)
        distances = {i: float('inf') for i in range(n)}
        distances[start] = 0
        parents = {i: None for i in range(n)}
        priority_queue = [(0, start)]

        while priority_queue:
            current_dist, current_node = heapq.heappop(priority_queue)

            if current_dist > distances[current_node]:
                continue

            for neighbor, weight in self.graph[current_node]:
                distance = current_dist + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    parents[neighbor] = current_node
                    heapq.heappush(priority_queue, (distance, neighbor))

        return distances, parents

    def reconstruct_path(self, parents, start, end):
        path = []
        current = end
        while current is not None:
            path.append(current)
            current = parents[current]
        return path[::-1]

    def generate_order_and_top(self, path):
        self.path_order = []
        self.path_top = []
        prev_node = None
        for i in range(len(path)):
            current_node = path[i]
            if prev_node is not None:
                direction = 1  # По умолчанию прямо
                self.path_order.append(f"L1 1")
                self.path_top.append(f"{prev_node} {current_node} {direction}")
            prev_node = current_node

    def optimize_route(self):
        self.build_graph()
        distances, parents = self.dijkstra(0)
        if distances[0] == float('inf'):
            return "No solution"

        path = self.reconstruct_path(parents, 0, len(self.route_points) - 1)
        self.generate_order_and_top(path)

        cost = 0
        for i, (x, y, value) in enumerate(self.route_points):
            if i == 0:
                continue
            di = distances[i]
            cost += value / (1 + di) if di != float('inf') else 0

        return -cost

    def write_output(self, cost):
        with open(self.output_path, 'w') as f:
            f.write(f"Optimized cost: {cost}\\n")
            f.write("ORDER\\n")
            for order in self.path_order:
                f.write(order + "\\n")
            f.write("/\\n")
            f.write("TOP\\n")
            for top in self.path_top:
                f.write(top + "\\n")
            f.write("/\\n")


if __name__ == "__main__":
    optimizer = RailRoadOptimizer("input.txt", "output.txt")
    optimizer.read_input()
    result = optimizer.optimize_route()
    if result == "No solution":
        print("No valid solution found.")
    else:
        optimizer.write_output(result)
        print(f"Optimized cost: {result}")
